In [1]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Convolution2D,Flatten,Dense,MaxPooling2D,Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

In [2]:
processing_data=np.load("D:\Machine Learning\AUTONOMOUS CAR\PROGRAM\final_models.npy")
ims=processing_data[0]
ster=processing_data[1]

x_train , x_test ,y_train , y_test =train_test_split(ims,ster,test_size=0.2)

In [3]:
train_image=[]
train_sterring=[]
test_image=[]
test_sterring=[]

for m in range(2000):
    train_image.append(x_train[m])
    train_sterring.append(y_train[m])

for n in range(1319):
    test_image.append(x_test[n])
    test_sterring.append(y_test[n])

del m
del n

# Augmentation

In [4]:
def processing(ims,ster):            
    x=[]
    y=[]

    for i in range(len(ims)):
        index=np.random.randint(0,len(ims)-1)
        lis=ims[index].split("\\")
        add=str("D:\Machine Learning\AUTONOMOUS CAR\DATA\IMG\\"+lis[5])
        image=cv.imread(add)
        steering_str=str(ster[index])
        steering_str=float(steering_str)

        image=image[60:135,:,:]

        image=cv.resize(image, (200,66))

        x.append(image)
        y.append(steering_str)

        if (steering_str > 0):
            image=cv.flip(image,1)
            steering=-float(ster[index])
            x.append(image)
            y.append(steering)

        else:
            image=cv.flip(image,1)
            steering=-float(ster[index])
            x.append(image)
            y.append(steering)

    x=np.array(x)
    y=np.array(y)

    return x,y

# CNN Model

In [144]:
model=Sequential()

model.add(Convolution2D(filters=16,kernel_size=(5,5),strides=(2,2),input_shape=(66,200,3),activation='relu',))
model.add(Convolution2D(filters=32,kernel_size=(5,5),strides=(2,2),activation='relu'))
model.add(Convolution2D(filters=64,kernel_size=(3,3),strides=(2,2),activation='relu'))
model.add(Convolution2D(filters=256,kernel_size=(3,3),strides=(2,2),activation='relu'))
model.add(Convolution2D(filters=256,kernel_size=(3,3),strides=(2,2),activation='relu',padding="SAME"))
model.add(Convolution2D(filters=512,kernel_size=(2,2),strides=(2,2),activation='relu',padding="SAME"))
model.add(Convolution2D(filters=512,kernel_size=(2,2),strides=(2,2),activation='relu',padding="SAME"))
model.add(Flatten())

model.add(Dense(units=100,activation="relu"))
model.add(Dense(units=50,activation="relu"))
model.add(Dense(units=20,activation="relu"))
model.add(Dense(units=10,activation="relu"))
model.add(Dense(units=1,kernel_regularizer=l2(0.01)))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [145]:
model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_266 (Conv2D)         (None, 31, 98, 16)        1216      
                                                                 
 conv2d_267 (Conv2D)         (None, 14, 47, 32)        12832     
                                                                 
 conv2d_268 (Conv2D)         (None, 6, 23, 64)         18496     
                                                                 
 conv2d_269 (Conv2D)         (None, 2, 11, 256)        147712    
                                                                 
 conv2d_270 (Conv2D)         (None, 1, 6, 256)         590080    
                                                                 
 conv2d_271 (Conv2D)         (None, 1, 3, 512)         524800    
                                                                 
 conv2d_272 (Conv2D)         (None, 1, 2, 512)       

# Process the data from Augmentation 

In [5]:
train_data_image,train_data_sterring=processing(train_image,train_sterring)
test_data_image,test_data_sterring=processing(test_image,test_sterring)

# Train model

In [147]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0.001) 
mc = ModelCheckpoint('Autosave.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [158]:
model.fit(train_data_image,train_data_sterring,steps_per_epoch=100,epochs =10,callbacks=[es,mc],batch_size=32)

Epoch 1/10
100/100 [==============================] - 25s 247ms/step - loss: -6.1743e-11 - accuracy: 0.7706
Epoch 2/10
100/100 [==============================] - 25s 247ms/step - loss: -7.3597e-10 - accuracy: 0.7866
Epoch 3/10
100/100 [==============================] - 25s 246ms/step - loss: 1.6632e-11 - accuracy: 0.8000
Epoch 4/10
100/100 [==============================] - 25s 249ms/step - loss: -1.9476e-10 - accuracy: 0.7884
Epoch 5/10
100/100 [==============================] - 25s 247ms/step - loss: 5.0755e-10 - accuracy: 0.7837
Epoch 6/10
100/100 [==============================] - 25s 249ms/step - loss: 3.5815e-10 - accuracy: 0.7691
Epoch 7/10
100/100 [==============================] - 25s 247ms/step - loss: -1.7445e-10 - accuracy: 0.7700
Epoch 8/10
100/100 [==============================] - 25s 246ms/step - loss: -2.5572e-10 - accuracy: 0.7744
Epoch 9/10
100/100 [==============================] - 25s 247ms/step - loss: -2.2104e-10 - accuracy: 0.7688
Epoch 10/10
100/100 [==========

# Save Model

In [153]:
model.save("Autonomous_car_model.h5",model)

# Model Prediction

In [142]:
lis=train_image[300].split("\\")
add=str("D:\Machine Learning\AUTONOMOUS CAR\DATA\IMG\\"+lis[5])

image=cv.imread(add)
image=cv.cvtColor(image,cv.COLOR_RGB2YUV)
image = cv.GaussianBlur(image,  (3, 3), 0)
image=image[60:135,:,:]
image = cv.resize(image, (200,66))
image = image/255.
image=np.array([image])
sterring_value=model.predict(image)
print(sterring_value)

1/1 [==============================] - 0s 35ms/step
[[-0.00134377]]
